##### Copyright 2018 The TensorFlow Authors.

In [0]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [0]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Regresi Dasar: Prediksi Efisiensi Bahan Bakar

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/keras/regression"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Lihat di TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/keras/regression.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Jalankan di Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/site/en/tutorials/keras/regression.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Lihat sumber kode di GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/site/en/tutorials/keras/regression.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Unduh notebook</a>
  </td>
</table>

Dalam permasalahan *regresi*, tujuan kita adalah untuk memprediksi keluaran berupa bilangan kontinu, seperti misalnya harga atau probabilitas. Berbeda dengan *klasifikasi*, dimana tujuannya adalah untuk memilih satu kelas dari beberapa kelas (contohnya, ketika terdapat gambar apel atau jeruk, menentukan buah apa yang terdapat di dalam gambar tersebut). 

Notebook ini menggunakan dataset klasik [Auto MPG](https://archive.ics.uci.edu/ml/datasets/auto+mpg) dan membuat model yang bertujuan untuk memprediksi efisiensi bahan bakar dari mobil yang dibuat pada akhir 1970-an dan awal 1980-an. Untuk melakukan hal ini, Model dengan deskripsi berbagai mobil pada periode itu telah disediakan. Deskripsi tersebut diantaranya adalah: jumlah silinder, *displacement*, *horsepower*, dan berat.

Kasus ini menggunakan API dari `tf.keras`, lihat [panduan ini](https://www.tensorflow.org/guide/keras) untuk lebih lengkap.

In [0]:
# Use seaborn for pairplot
!pip install seaborn

# Use some functions from tensorflow_docs
!pip install git+https://github.com/tensorflow/docs

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import pathlib

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [0]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)

In [0]:
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

## Dataset Auto MPG

Dataset dapat diperoleh dari [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/).


### Memperoleh Data
Pertama-tama, unduh dataset terlebih dahulu.

In [0]:
dataset_path = keras.utils.get_file("auto-mpg.data", "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")
dataset_path

Import dataset menggunakan pandas

In [0]:
column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight',
                'Acceleration', 'Model Year', 'Origin']
raw_dataset = pd.read_csv(dataset_path, names=column_names,
                      na_values = "?", comment='\t',
                      sep=" ", skipinitialspace=True)

dataset = raw_dataset.copy()
dataset.tail()

### Membersihkan data

Dataset masih mengandung beberapa *missing value*.

In [0]:
dataset.isna().sum()

Untuk mempermudah tutorial ini, baris-baris yang mengandung nilai kosong tersebut akan di drop.

In [0]:
dataset = dataset.dropna()

Kolom `"Origin"` sebenarnya berisikan data kategorikal, bukan numerik. Oleh karena itu, ubah kolom tersebut dengan metode enkode one-hot:

In [0]:
dataset['Origin'] = dataset['Origin'].map(lambda x: {1: 'USA', 2: 'Europe', 3: 'Japan'}.get(x))

In [0]:
dataset = pd.get_dummies(dataset, prefix='', prefix_sep='')
dataset.tail()

### Membagi data menjadi data train dan data test

Sekarang bagi dataset menjadi set untuk training dan set untuk tes.

Kita akan menggunakan set untuk tes pada evaluasi akhir model kita.

In [0]:
train_dataset = dataset.sample(frac=0.8,random_state=0)
test_dataset = dataset.drop(train_dataset.index)

### Inspeksi data

Melihat *joint distribution* dari beberapa kolom yang terdapat di data training.

In [0]:
sns.pairplot(train_dataset[["MPG", "Cylinders", "Displacement", "Weight"]], diag_kind="kde")

Juga melihat keseluruhan statistik dari data:

In [0]:
train_stats = train_dataset.describe()
train_stats.pop("MPG")
train_stats = train_stats.transpose()
train_stats

### Memisahkan fitur dari label

Pisahkan nilai target, atau "label", dari fitur. Label ini adalah nilai yang akan diprediksi oleh model.

In [0]:
train_labels = train_dataset.pop('MPG')
test_labels = test_dataset.pop('MPG')

### Normalisasi data

Lihat kembali `train_stats` di atas dan perhatikan perbedaan nilai range untuk setiap fitur yang ada.

Melakukan normalisasi terhadap fitur yang memiliki skala dan range yang berbeda merupakan praktik yang baik dalam membangun model. Meskipun model *mungkin* dapat dibuat tanpa melakukan normalisasi fitur terlebih dahulu, hal ini akan membuat proses training data menjadi lebih sulit, dan akan membuat model yang dihasilkan terlalu dipengaruhi oleh unit input.

Catatan: Meskipun kita sengaja membangun fungsi statistik ini dengan menggunakan data training saja, fungsi ini juga akan digunakan untuk menormalisasi data tes. Kita perlu melakukan hal tersebut agar data tes memiliki bentuk yang sama dengan data yang digunakan untuk melatih model.

In [0]:
def norm(x):
  return (x - train_stats['mean']) / train_stats['std']
normed_train_data = norm(train_dataset)
normed_test_data = norm(test_dataset)

Data yang sudah dinormalisasi ini akan digunakan sebagai data untuk membuat model.

Perhatian: nilai statistik yang digunakan untuk normalisasi input (rata-rata dan standard deviasi) harus digunakan juga pada data yang akan dimasukan ke dalam model, termasuk juga proses enkode one-hot yang telah kita lakukan sebelumnya. Hal ini berlaku juga untuk data tes dan data lainnya ketika hendak menggunakan model pada fase produksi.

## Model

### Membangun Model

Mari kita bangun model kita. Disini, kita akan menggunakan model `Sequential` dengan dua *hidden layer* yang terhubung secara penuh satu sama lain, dan satu layer output yang akan memberikan hasil berupa bilangan kontinu. Langkah-langkah pembuatan model akan dimasukkan dalam sebuah fungsi, `build_model`. Hal ini dilakukan karena nantinya kita akan membuat model yang lainnya.

In [0]:
def build_model():
  model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(train_dataset.keys())]),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

In [0]:
model = build_model()

### Inspeksi Model

Gunakan *method* `.summary` untuk memunculkan deskripsi sederhana dari model.

In [0]:
model.summary()

Sekarang uji coba model yang telah dibuat. Ambil `10` nilai dari data training dan panggil `model.predict` pada model kita.

In [0]:
example_batch = normed_train_data[:10]
example_result = model.predict(example_batch)
example_result

Model dapat berfungsi dan menghasilkan nilai dengan bentuk dan tipe yang diharapkan.

### Proses train dari model

Latih model dengan 1000 epoch, dan catat nilai akurasi dari training dan validasi dalam objek `history`

In [0]:
EPOCHS = 1000

history = model.fit(
  normed_train_data, train_labels,
  epochs=EPOCHS, validation_split = 0.2, verbose=0,
  callbacks=[tfdocs.modeling.EpochDots()])

Visualisasi progres pelatihan model menggunakan nilai-nilai yang disimpan dalam objek `history`.

In [0]:
hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

In [0]:
plotter = tfdocs.plots.HistoryPlotter(smoothing_std=2)

In [0]:
plotter.plot({'Basic': history}, metric = "mae")
plt.ylim([0, 10])
plt.ylabel('MAE [MPG]')

In [0]:
plotter.plot({'Basic': history}, metric = "mse")
plt.ylim([0, 20])
plt.ylabel('MSE [MPG^2]')

Grafik ini menunjukan sedikit peningkatan, atau penurunan nilai error dari validasi setelah sekitar 100 epoch. Mari kita perbarui pemanggilan `model.fit` agar proses pelatihan model dapat berhenti secara otomatis ketika skor validasi sudah tidak meningkat lagi. Kita akan menggunakan *EarlyStopping callback* yang akan mengecek kondisi dari proses pelatihan model setiap epoch. Apabila beberapa kali epoch dilalui tanpa menunjukkan peningkatan, maka proses pelatihan akan berhenti secara otomatis. 

Anda dapat mempelajari lebih lanjut tentang *callback* [disini](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/EarlyStopping).

In [0]:
model = build_model()

# The patience parameter is the amount of epochs to check for improvement
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

early_history = model.fit(normed_train_data, train_labels, 
                    epochs=EPOCHS, validation_split = 0.2, verbose=0, 
                    callbacks=[early_stop, tfdocs.modeling.EpochDots()])

In [0]:
plotter.plot({'Early Stopping': early_history}, metric = "mae")
plt.ylim([0, 10])
plt.ylabel('MAE [MPG]')

Grafik menunjukan bahwa pada set validasi, rata-rata error berkisar antara +/- 2 MPG. Apakah ini bagus? Kita akan menyerahkan jawaban tersebut kepada Anda.

Mari kita lihat sebarapa baik performa model dengan menggunakan data **tes**, yaitu data yang tidak kita gunakan dalam proses pelatihan model. Hal ini menujukkan kepada kita seberapa baik performa dari model ketika kita menggunakannya di dunia nyata.

In [0]:
loss, mae, mse = model.evaluate(normed_test_data, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} MPG".format(mae))

### Membuat prediksi

Akhirnya, kita akan melakukan prediksi nilai MPG menggunakan test data:

In [0]:
test_predictions = model.predict(normed_test_data).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True Values [MPG]')
plt.ylabel('Predictions [MPG]')
lims = [0, 50]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims, lims)


Sepertinya model kita dapat memprediksi dengan cukup baik. Mari kita lihat distribusi error dari prediksi tersebut.

In [0]:
error = test_predictions - test_labels
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error [MPG]")
_ = plt.ylabel("Count")

Nilainya tidak terdistribusi secara normal, tetapi hal ini wajar karena jumlah sampel yang digunakan sedikit.

## Kesimpulan

Notebook ini memperkenalkan beberapa teknik untuk menyelesaikan permasalahan regresi.

* Mean Squared Error (MSE) adalah *loss function* yang biasa digunakan dalam permasalahan regresi (*loss function* yang berbeda digunakan pada kasus klasifikasi).
* Metriks evaluasi yang digunakan untuk regresi juga berbeda dengan yang digunakan untuk klasifikasi. Metriks yang umum untuk regresi adalah Mean Absolute Error (MAE).
* Ketika fitur data input yang bertipe numerik memiliki range yang berbeda, setiap fitur harus diubah nilainya terlebih dahulu sedemikian sehingga fitur-fitur tersebut memiliki nilai dengan range yang sama.
* Apabila data training yang dimiliki tidak banyak, lebih baik menggunakan *network* yang sederhana dengan sedikit *hidden layer* untuk menghindari *overfitting*.
* *Early stopping* merupakan teknik yang bermanfaat untuk menghindari *overfitting*.